In [1]:
import numpy as np
from os.path import join
def get_node_set(path):
    # training data
    edges_unordered = np.genfromtxt(path,
                                    dtype=np.int32)
    id_set = set(edges_unordered.flatten().tolist())
    return id_set

data_path = join('./','kaggle')
# emb
with open(join('./','t1.emb')) as f:

    num_nodes, D = f.readline().strip().split(' ')
    num_nodes = int(num_nodes)
    D = int(D)
    
    ls = f.readlines()
node_emb_dict = {}
for l in ls:
    buf = l.strip().split(' ')
    node_id, emb = int(buf[0]), buf[1:]
    x = np.asarray([float(i) for i in emb], dtype=np.float32)
    node_emb_dict[node_id] = x
    
# training data
with open(join(data_path,'t1-merge.txt')) as f:
    ls = f.readlines()
node_set = get_node_set(join(data_path,'t1-merge.txt'))
idx_map = {k:i for i,k in enumerate(list(node_set))}
N = len(node_set)
X = []
adj_mat = np.zeros([N,N], dtype=np.uint8)
for l in ls:
    buf = l.strip().split(' ')
    src, dst = int(buf[0]), int(buf[1])
    adj_mat[idx_map[src], idx_map[dst]] = 1
    fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
    X.append(fea)
X = np.vstack(X)

# test data
with open(join(data_path,'t1-test.txt')) as f:
    ls = f.readlines()
N2 = len(ls)
test_X = []
for i,l in enumerate(ls):
    buf = l.strip().split(' ')
    src, dst = int(buf[0]), int(buf[1])
    
    if src not in node_emb_dict:
        src = 6188
        dst = 31366
    if dst not in node_emb_dict:
        src = 6188
        dst = 31366
    fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
    
    test_X.append(fea)
test_X = np.vstack(test_X)
print(X.shape, test_X.shape)
print 'done'
    

((285789, 256), (88074, 256))
done


In [2]:
import numpy as np
batch_size = 128
def naive_bootsrap_generator(X, adj_mat, idx_map, node_emb_dict, train_node_set, batch_size=128, neg_rate=1. ):
    train_node_list = list(train_node_set)
    train_N = len(train_node_list)
    num_edge = X.shape[0]
        
    while True:
        idx = np.random.choice(num_edge, batch_size)
        pos_X = X[idx, :]
        
        neg_count = int(batch_size*neg_rate)
        neg_idx = np.random.randint(train_N, size=[neg_count, 2])
        neg_X = []
        for i in range(neg_count):
            src, dst = neg_idx[i]
            src = train_node_list[src]
            dst = train_node_list[dst]
            if src != dst and adj_mat[idx_map[src], idx_map[dst]] == 0:
                fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
                neg_X.append(fea)
        neg_X = np.vstack(neg_X)

        ret_X = np.vstack([pos_X, neg_X])
        ret_Y = np.zeros([ret_X.shape[0], 1])
        ret_Y[:batch_size, 0] = 1
        yield ret_X, ret_Y

N = X.shape[0]
idx = np.random.permutation(N)
train_idx = idx[N//10:]
val_idx = idx[:N//10]

train_X = X[train_idx,:]
val_X = X[val_idx,:]

train_node_set = get_node_set('./kaggle/t1-train.txt')
G = naive_bootsrap_generator(train_X, adj_mat, idx_map, node_emb_dict, train_node_set, batch_size=batch_size)
val_G = naive_bootsrap_generator(val_X, adj_mat, idx_map, node_emb_dict, train_node_set,batch_size=batch_size, neg_rate=0.1)
x,y = next(G)
print x.shape,y.shape
x,y = next(val_G)
print x.shape,y.shape

(256, 256) (256, 1)
(140, 256) (140, 1)


In [3]:
import keras
from keras.models import *
from keras.layers import *

epochs = 100
def build_model():

    model = Sequential()
    model.add(Dense(256, activation='selu', input_shape=(256,)))
    model.add(Dense(256, activation='selu'))
    model.add(Dropout(0.1))
    model.add(Dense(128, activation='selu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='selu'))
    model.add(Dropout(0.1))
    model.add(Dense(32, activation='selu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation='selu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])
    return model
x,y = next(G)
print x.shape, y.shape
np.random.seed(1337)
model = build_model()
model.fit(x,y)
ck = keras.callbacks.ModelCheckpoint('./weights.hdf5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
tfb = keras.callbacks.TensorBoard(log_dir='./logs')
model.fit_generator(G,
                    steps_per_epoch=train_X.shape[0]//batch_size,
                    epochs=1000, verbose=1,
                    validation_data=val_G,
                    validation_steps=val_X.shape[0]//batch_size,
                    callbacks=[ck,tfb]
                    )



Using TensorFlow backend.


(256, 256) (256, 1)
Epoch 1/1
256/256 [==============================] - 1s 3ms/step - loss: 0.9252 - acc: 0.5430
Epoch 1/1000
2009/2009 [==============================] - 9s 4ms/step - loss: 0.3847 - acc: 0.8205 - val_loss: 0.2580 - val_acc: 0.8988
Epoch 2/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.2668 - acc: 0.8902 - val_loss: 0.1711 - val_acc: 0.9381
Epoch 3/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.2185 - acc: 0.9130 - val_loss: 0.1955 - val_acc: 0.9232
Epoch 4/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.1857 - acc: 0.9279 - val_loss: 0.1372 - val_acc: 0.9506
Epoch 5/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.1659 - acc: 0.9362 - val_loss: 0.1224 - val_acc: 0.9597
Epoch 6/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.1489 - acc: 0.9437 - val_loss: 0.1197 - val_acc: 0.9564
Epoch 7/1000
2009/2009 [==============================] - 8s 4ms/step 

2009/2009 [==============================] - 8s 4ms/step - loss: 0.0419 - acc: 0.9865 - val_loss: 0.0824 - val_acc: 0.9764
Epoch 61/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0425 - acc: 0.9863 - val_loss: 0.1124 - val_acc: 0.9655
Epoch 62/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0418 - acc: 0.9867 - val_loss: 0.0929 - val_acc: 0.9753
Epoch 63/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0417 - acc: 0.9868 - val_loss: 0.1012 - val_acc: 0.9715
Epoch 64/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0403 - acc: 0.9873 - val_loss: 0.1055 - val_acc: 0.9717
Epoch 65/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0403 - acc: 0.9872 - val_loss: 0.0877 - val_acc: 0.9742
Epoch 66/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0403 - acc: 0.9870 - val_loss: 0.1398 - val_acc: 0.9660
Epoch 67/1000
2009/2009 [==============================

Epoch 120/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0323 - acc: 0.9902 - val_loss: 0.1118 - val_acc: 0.9727
Epoch 121/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0319 - acc: 0.9904 - val_loss: 0.0970 - val_acc: 0.9736
Epoch 122/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0307 - acc: 0.9907 - val_loss: 0.1257 - val_acc: 0.9700
Epoch 123/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0317 - acc: 0.9904 - val_loss: 0.1262 - val_acc: 0.9666
Epoch 124/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0317 - acc: 0.9905 - val_loss: 0.0842 - val_acc: 0.9759
Epoch 125/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0322 - acc: 0.9903 - val_loss: 0.1080 - val_acc: 0.9727
Epoch 126/1000
2009/2009 [==============================] - 8s 4ms/step - loss: 0.0320 - acc: 0.9904 - val_loss: 0.1351 - val_acc: 0.9715
Epoch 127/1000
2009/2009 [========

KeyboardInterrupt: 

In [5]:
from keras.models import *
model = load_model('./weights.hdf5')

z = model.predict(test_X)
with open('pred.txt', 'w') as f:
    for i in range(z.shape[0]):
        p = z[i,0]
        ans = 1 if p >= 0.5 else 0
        f.write('%d\n' % ans)
pred_file = 'pred.txt'
with open(pred_file, 'r') as f, open(pred_file + '.csv', 'w') as g:
    g.write('query_id,prediction\n')
    for idx, line in enumerate(f):
        g.write('%d,%d\n' % (1 + idx, int(line)))
#         print idx
        


print 'done'
print z.shape

1
done
(88074, 1)


### SVM version

In [2]:
from svm import *
from svmutil import *
# prepare data for SVM
def generate_neg_X(X, adj_mat, node_emb_dict,neg_rate=1.5):
    exist_node_list = node_emb_dict.keys()
    exist_N = len(exist_node_list)
    num_edge = X.shape[0]
        
    pos_X = X

    neg_count = int(num_edge*neg_rate)
    neg_idx = np.random.randint(exist_N, size=[neg_count, 2])
    neg_X = []
    for i in range(neg_count):
        src, dst = neg_idx[i]
        src = exist_node_list[src]
        dst = exist_node_list[dst]
        if src != dst and adj_mat[src, dst] == 0:
            fea = np.concatenate([node_emb_dict[src], node_emb_dict[dst]], axis=-1)
            neg_X.append(fea)
    neg_X = np.vstack(neg_X)

    ret_X = np.vstack([pos_X, neg_X])
    ret_Y = np.ones([ret_X.shape[0], 1])
    ret_Y[pos_X.shape[0]:, 0] = -1
    return ret_X, ret_Y
train_X, train_Y = generate_neg_X(X, adj_mat, node_emb_dict,neg_rate=1.5)
print train_X.shape
prob = svm_problem(train_Y.flatten(),train_X)
print 'prob done'

(714296, 256)
prob done


In [ ]:
param = svm_parameter('-s 0 -t 2 -m 3000')
m = svm_train(prob, param)
p_labels, p_acc, p_vals = svm_predict([], test_X, m)

### Fake Link Generation

In [2]:
# randomly sample test link
import numpy as np
from tqdm import tqdm
from os.path import join
def get_node_set(path):
    # training data
    edges_unordered = np.genfromtxt(path,
                                    dtype=np.int32)
    id_set = set(edges_unordered.flatten().tolist())
    return id_set

data_path = join('./','kaggle')
# training data
train_node_set = get_node_set(join(data_path,'t1-train.txt'))
test_node_set = get_node_set(join(data_path,'t1-test-seen.txt'))
node_set = set.union(train_node_set, test_node_set)
idx_map = {k:i for i,k in enumerate(list(node_set))}
N = len(node_set)
adj_mat = np.zeros([N,N], dtype=np.uint8)

links = np.genfromtxt(join(data_path,'t1-merge.txt'), dtype=np.int32)
for i in range(links.shape[0]):
    
    src, dst = links[i].tolist()
    adj_mat[idx_map[src], idx_map[dst]] = 1

out_degree = np.sum(adj_mat, axis=1).flatten()
rev_map = {v:k for k,v in idx_map.items()}
total_link_num = links.shape[0] + int(np.sum(out_degree))
with tqdm(total=total_link_num) as pbar:
    with open(join(data_path,'t1-fake.txt'), 'w') as f:
        for i in range(links.shape[0]):
            src, dst = links[i].tolist()
            s = '%d %d\n' % (src, dst)
            f.write(s)
            pbar.update(1)
        train_node_list = list(train_node_set)
        for node_id in list(test_node_set):
            i = idx_map[node_id]
            d = out_degree[i]
            
            for j in range(d):
                idx = np.random.randint(len(train_node_list))
                dst = idx_map[train_node_list[idx]]
                while adj_mat[i, dst] == 1 or dst == i:
                    idx = np.random.randint(len(train_node_list))
                    dst = idx_map[train_node_list[idx]]
                
                adj_mat[i, dst] = 1
                s = '%d %d\n' % (rev_map[i], rev_map[dst])
                f.write(s)
            
                pbar.update(1)
    
print 'done', np.sum(adj_mat)
    

 83%|████████▎ | 475946/571578 [00:01<00:00, 360325.21it/s]


done 475946


In [23]:
# GCN version
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F
from layers import GraphConvolution
import time
import torch.optim as optim



def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot


def load_data(path, idx_map):
    print('Loading from file {} ...'.format(path))


    # build graph
    edges_unordered = np.genfromtxt(path,
                                    dtype=np.int32)
    N = len(idx_map)
    
#     print edges_unordered.shape
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
#     edges = np.vstack([edges, np.flip(edges, axis=1)])
#     print edges.shape
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(N, N),
                        dtype=np.float32)
    src_adj = adj
    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    adj = normalize(adj + sp.eye(adj.shape[0]))

    adj = sparse_mx_to_torch_sparse_tensor(adj)

    return src_adj, adj


def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)
def accuracy_mse(output, labels):
    correct = torch.abs(output - labels) < 0.5
    return torch.sum(correct).item() / len(labels)
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)


# print adj.shape




class GCN(nn.Module):
    def __init__(self, node_num, nhid, nclass, dropout_rate):
        super(GCN, self).__init__()
#         self.emb = nn.Embedding(node_num, nhid)
        self.gc1 = GraphConvolution(nhid, nhid)
        self.mid = GraphConvolution(nhid, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x, adj):
#         x = self.emb(x)
        x = F.selu(self.gc1(x, adj))
        x = self.dropout(x)
        l1 = x = F.selu(self.mid(x, adj))
        
        x = self.dropout(x)
        x = self.gc2(x, adj)
#         return F.log_softmax(x, dim=1)
        return x, l1


def count_degree(adj):
    node_num = adj.shape[0]
    return 
def count_neighbor(adj):
    N = adj.shape[0]
    m = {i:[] for i in range(N)}
    rows, cols, values = sp.find(adj)
    for src,dst,v in zip(rows, cols, values):
        if v == 1:
            m[src].append(dst)
            m[dst].append(src)
    ret = []
    for i in range(N):
        neighbor_set = set(m[i])
        ret.append(len(neighbor_set))
    return np.array(ret).reshape([-1, 1])
def sparse_diag(degree):
    v = degree.flatten()
    N = v.size()[0]
    i = torch.cat([torch.arange(N).view([1,-1]), torch.arange(N).view([1,-1])], dim=0)
    sp = torch.sparse.FloatTensor(i, v, torch.Size([N,N]))
    return sp

# Load data
adj, sym_adj = load_data('./kaggle/t1-fake.txt', idx_map)
node_num = adj.shape[0]


in_degree = np.sum(adj, axis=0).flatten()
out_degree = np.sum(adj, axis=1).flatten()
neighbor_count = count_neighbor(adj)
degree = in_degree + out_degree
second_order_adj = np.dot(adj, adj)
second_order_degree = np.sum(second_order_adj, axis=0).flatten() + np.sum(second_order_adj, axis=1).flatten()


degree = degree.reshape([-1, 1])

second_order_degree = second_order_degree.reshape([-1, 1])
idx = np.random.permutation(node_num)
train_idx, val_idx = idx[:node_num//10], idx[node_num//10:]

# 
# features = torch.LongTensor(features)
rev_map = {v:k for k,v in idx_map.items()}
features = torch.FloatTensor(np.vstack([node_emb_dict[rev_map[i]] for i in range(node_num)]))
# features = torch.arange(node_num)

degree = torch.FloatTensor(degree)
out_degree = torch.FloatTensor(out_degree)
label_neighbor_count = torch.FloatTensor(neighbor_count)

# labels = label_neighbor_count
labels = out_degree
# labels = torch.FloatTensor(in_degree.reshape([-1,1]))
# Model and optimizer
model = GCN(
            node_num=node_num,
            nhid=128,
            nclass=1,
            dropout_rate=0.15)
optimizer = optim.Adam(model.parameters(),
                       lr=0.0087)
criterion = nn.MSELoss()
l1_criterion = nn.L1Loss()
# 
model.cuda()
features = features.cuda()
sym_adj = sym_adj.cuda()
labels = labels.view([-1,1]).cuda()

reg_adj = sparse_diag(degree).cuda()
print features.shape, node_num
# 
t = time.time()
    
for i in range(1,1000):
        
    optimizer.zero_grad()
    model.train()
    output,_ = model(features, sym_adj)
#     print output.size(), labels.size(),  'out'
    loss_train = criterion(output[train_idx,:], labels[train_idx,:] )
#     gcn_reg_loss = torch.sum( torch.mm(output.t(), torch.spmm(reg_adj, output)) )
#     l1_loss = l1_criterion(l1, torch.zeros_like(l1).cuda())
#     l2_loss = criterion(l1, torch.zeros_like(l1).cuda())
#     loss_train = criterion(output[train_idx,0], labels_1[train_idx,0])
#     a = 0.000
#     (loss_train + a*gcn_reg_loss).backward()
    (loss_train).backward()
    optimizer.step()
#     break
#     val
    model.eval()
    loss_val = criterion(output[val_idx,:], labels[val_idx,:]) 

    acc1 = accuracy_mse(output[val_idx,:], labels[val_idx,:])
    if i % 10 ==0:
#         print( a*gcn_reg_loss.item())
        print('Epoch: {:04d}'.format(i+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc1_val: {:.4f}'.format(acc1),
              'time: {:.4f}s'.format((time.time() - t)))

# Train model
print("Optimization Finished!")
# print("Total time elapsed: {:.4f}s".format(time.time() - t_total))




Loading from file ./kaggle/t1-fake.txt ...
torch.Size([29402, 128]) 29402
('Epoch: 0011', 'loss_train: 546.4982', 'loss_val: 527.1210', 'acc1_val: 0.0000', 'time: 1.9512s')
('Epoch: 0021', 'loss_train: 517.0976', 'loss_val: 503.0954', 'acc1_val: 0.0000', 'time: 3.8818s')
('Epoch: 0031', 'loss_train: 518.0991', 'loss_val: 505.3470', 'acc1_val: 0.0000', 'time: 5.8127s')
('Epoch: 0041', 'loss_train: 516.2365', 'loss_val: 501.8302', 'acc1_val: 0.0000', 'time: 7.7438s')
('Epoch: 0051', 'loss_train: 514.1308', 'loss_val: 500.7548', 'acc1_val: 0.0000', 'time: 9.6746s')
('Epoch: 0061', 'loss_train: 511.6733', 'loss_val: 499.1846', 'acc1_val: 0.0000', 'time: 11.6054s')
('Epoch: 0071', 'loss_train: 507.1001', 'loss_val: 495.6954', 'acc1_val: 0.0000', 'time: 13.5363s')
('Epoch: 0081', 'loss_train: 501.9738', 'loss_val: 494.1317', 'acc1_val: 0.0000', 'time: 15.4673s')
('Epoch: 0091', 'loss_train: 498.0366', 'loss_val: 494.4380', 'acc1_val: 0.0000', 'time: 17.3982s')
('Epoch: 0101', 'loss_train: 49

KeyboardInterrupt: 

In [24]:
# output GCN emb
model.eval()
with torch.no_grad():
    output, l1 = model(features, sym_adj)
l1 = l1.cpu().numpy()
rev_map = {v:k for k,v in idx_map.items()}
with open('GCN.emb','w' ) as f:
    f.write('%d %d\n' % (node_num, l1.shape[1]) )
    for i in range(l1.shape[0]):
        f.write('%d' % rev_map[i])
        for x in l1[i,:].flatten().tolist():
            f.write(' %f' % x)
        f.write('\n')
print 'done'
        


done


In [14]:
a = torch.tensor([1,2,3])


tensor([[1, 0, 0],
        [0, 2, 0],
        [0, 0, 3]])